In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

# Input data files are available in the read-only "../input/" directory
# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np  # linear algebra
import math
from sklearn import svm
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from gensim import corpora, models, similarities

# Data Loading
main_path = 'E:/PycharmProjects/EE448/ee448-project2-query-expansion/'

doc_dict = dict()
query_dict = dict()
rele_dict = dict()
# doc.txt
with open(main_path+'doc.txt') as lines:
    for line in lines:
        id, text = line.strip().split('\t')
        doc_dict[id] = text
# query.txt
with open(main_path+'query.txt') as lines:
    for line in lines:
        id, text = line.strip().split('\t')
        query_dict[id] = text
# rele.txt
with open(main_path+'rele.txt') as lines:
    for line in lines:
        query_id, document_id, relevance_score = line.strip().split('\t')
        if rele_dict.get(query_id) == None:
            rele_dict[query_id] = dict()
        rele_dict[query_id][document_id] = relevance_score

In [3]:
# calc bm25
def example_list2dict(input):
    output = dict()
    for word in input.split():
        if output.get(word) is None:
            output[word] = 0
        output[word] += 1
    return output


def cal_idf(doc_dict):
    doc_num = len(doc_dict)
    idf = dict()
    for doc_id in doc_dict:
        doc_text = list(set(doc_dict[doc_id].split()))
        for word in doc_text:
            if idf.get(word) is None:
                idf[word] = 0
            idf[word] += 1
    for word in idf:
        idf[word] = math.log((doc_num - idf[word] + 0.5) / (idf[word] + 0.5))
    return idf


def bm25(query, doc, idf, avg_doc_len=374):
    k1 = 2.0
    k2 = 1.0
    b = 0.75
    score = 0.0
    for word in query:
        if doc.get(word) == None:
            continue
        W_i = idf[word]
        f_i = doc[word]
        qf_i = query[word]
        doc_len = sum(doc.values())
        K = k1 * (1 - b + b * doc_len / avg_doc_len)
        R1 = f_i * (k1 + 1) / (f_i + K)
        R2 = qf_i * (k2 + 1) / (qf_i + k2)
        R = R1 * R2
        score += W_i * R
    return score

In [4]:
def doc_sim(query, corpus):
    """
    Get  TF-IDF values between query and corpus

    Args:
        query (list/Series): query in the form of a nested list or pandas.Series
        corpus (list): documents in the form of a nested list or pandas.Series
     Returns:
        sims: similarity scores
    """
    dictionary = corpora.Dictionary(corpus)
    
    doc_vectors = [dictionary.doc2bow(text) for text in corpus]
    
    tfidf = models.TfidfModel(doc_vectors)
    
    tfidf_vectors = tfidf[doc_vectors]
    
    query_bow = dictionary.doc2bow(query)
    
    index = similarities.MatrixSimilarity(tfidf_vectors)
    
    sims = index[query_bow]
    
    return sims


def top_tfidf(docs, n, n_gram=(1, 1)):
    """
    Get top n terms with highest TF-IDF value
    from some docs.

    Args:
        docs (list/Series): Text document in the form of a nested list or pandas.Series
        n (int): Number of terms. To get all possible terms, put n=-1
        n_gram (tuple): Range of n_gram, default (1,1)
     Returns:
        top_term_with_scores: returns top n terms with the highest TF-IDF values,
        together with the TF-IDF values
    """
    # initialize TF-IDF vectorizer
    vectorizer = TfidfVectorizer(ngram_range=n_gram)

    # Fit vectorizer to the documents
    vectorizer.fit(docs)

    # Get TF-IDF values from the documents
    transformed = vectorizer.transform(docs)

    # Get the TF-IDF value for each word/vocabulary
    scores = zip(vectorizer.get_feature_names(), np.asarray(transformed.sum(axis=0)).ravel())

    # Sort terms based on highest TF-IDF value
    top_term_with_scores = sorted(scores, key=lambda x: x[1], reverse=True)

    # Return top n terms with the highest TF-IDF value
    top_term = []
    for item in top_term_with_scores[:n]:
        top_term.append(item[0])
    return top_term

In [5]:
def calc_freq(terms, doc, window_size):
    """
    Args:
        terms: str list
        doc: str list
        window_size: int
    Returns:
        count: int
    """
    term_num = len(terms)
    doc_len = len(doc)
    count = 0
    for index in range(0, doc_len - window_size + 1):
        flag = True
        for i in range(0, term_num):
            if not (terms[i] in doc[index:index + window_size]):
                flag = False
        if flag:
            count += 1
    return count


def min_dist(termA, termB, doc):
    """
    Args:
        termA: str
        termB: str
        doc: str list
    Returns:
        min_dist: int
    """
    minD = len(doc)
    posA = posB = -1
    for index in range(0, len(doc)):
        if doc[index] == termA:
            posA = index
        elif doc[index] == termB:
            posB = index
        if posA > -1 and posB > -1 and minD > abs(posA - posB):
            minD = abs(posA - posB)
    return minD


def ave_prec(rec, prec, use_07_metric=False):
    """
    Args:
        rec: recall list
        prec: precision list
    Returns:
        ap: average precision
    """
    if use_07_metric:
        # 11 point metric
        ap = 0.
        for t in np.arange(0., 1.1, 0.1):
            if np.sum(rec >= t) == 0:
                p = 0
            else:
                p = np.max(prec[rec >= t])
            ap = ap + p / 11.
    else:
        mrec = np.concatenate(([0.], rec, [1.]))
        mpre = np.concatenate(([0.], prec, [0.]))
        for i in range(mpre.size - 1, 0, -1):
            mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])
        i = np.where(mrec[1:] != mrec[:-1])[0]
        ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
    return ap

In [6]:
# used for calc term's features and label
def get_feature(term, query_id, doc_ids, releDoc_ids):
    '''
    Args:
        term: str
        query_id: str
        doc_ids: str list
        releDoc_ids: str list
    Returns:
        feature: float list (1x8)
    '''
    global query_dict, doc_dict
    features = [0, 0, 0, 0, 0, 0, 0, 0]

    query = query_dict[query_id].split()

    min_dist_all = []
    term_fre_all = []
    query_fre_all = []
    Co_occur_fre_all = []
    pair_occur_fre_all = []

    min_dist_rele = []
    term_fre_rele = []
    query_fre_rele = []
    Co_occur_fre_rele = []
    pair_occur_fre_rele = []

    for doc_id in doc_ids:
        tmp_dist = []
        tmp_query = []
        tmp_Co_occur = []
        tmp_pair_occur = []

        for m in range(0, len(query)):
            tmp_dist.append(min_dist(query[m], term, doc_dict[doc_id].split()))
            tmp_query.append(calc_freq([query[m]], doc_dict[doc_id].split(), 1))
            tmp_Co_occur.append(calc_freq([query[m], term], doc_dict[doc_id].split(), 12))
            for n in range(m+1, len(query)):
                tmp_pair_occur.append(calc_freq([query[m], query[n], term], doc_dict[doc_id].split(), 15))

        if not min_dist_all:
            min_dist_all = tmp_dist[:]
        else:
            for i in range(0, len(min_dist_all)):
                if min_dist_all[i] > tmp_dist[i]:
                    min_dist_all[i] = tmp_dist[i]
        term_fre_all.append(calc_freq([term], doc_dict[doc_id].split(), 1))
        query_fre_all.append(tmp_query)
        Co_occur_fre_all.append(tmp_Co_occur)
        pair_occur_fre_all.append(tmp_pair_occur)

        if doc_id in releDoc_ids:
            if not min_dist_rele:
                min_dist_rele = tmp_dist[:]
            else:
                for i in range(0, len(min_dist_rele)):
                    if min_dist_rele[i] > tmp_dist[i]:
                        min_dist_rele[i] = tmp_dist[i]
            term_fre_rele.append(term_fre_all[-1])
            query_fre_rele.append(query_fre_all[-1])
            Co_occur_fre_rele.append(Co_occur_fre_all[-1])
            pair_occur_fre_rele.append(pair_occur_fre_all[-1])

    min_dist_all = np.array(min_dist_all)
    term_fre_all = np.array(term_fre_all)
    query_fre_all = np.array(query_fre_all)
    Co_occur_fre_all = np.array(Co_occur_fre_all)
    pair_occur_fre_all = np.array(pair_occur_fre_all)

    min_dist_rele = np.array(min_dist_rele)
    term_fre_rele = np.array(term_fre_rele)
    query_fre_rele = np.array(query_fre_rele)
    Co_occur_fre_rele = np.array(Co_occur_fre_rele)
    pair_occur_fre_rele = np.array(pair_occur_fre_rele)

    # Term distributions
    features[0] = math.log(1 + term_fre_rele.sum() / query_fre_rele.sum(), 10)
    features[1] = math.log(1 + term_fre_all.sum() / query_fre_all.sum(), 10)
    # Co-occurrence with single query term
    features[2] = math.log(1 + Co_occur_fre_rele.sum() / (len(query) * query_fre_rele.sum()), 10)
    features[3] = math.log(1 + Co_occur_fre_all.sum() / (len(query) * query_fre_all.sum()), 10)
    # Co-occurrence with pairs query terms
    features[4] = math.log(1 + pair_occur_fre_rele.sum() / (pair_occur_fre_rele.shape[1] * query_fre_rele.sum() + 0.001), 10)
    features[5] = math.log(1 + pair_occur_fre_all.sum() / (pair_occur_fre_all.shape[1] * query_fre_all.sum() + 0.001), 10)
    # Weighted term proximity
    features[6] = math.log(1 + np.sum(Co_occur_fre_rele.sum(axis=0) * min_dist_rele) / (Co_occur_fre_rele.sum() + 0.001), 10)
    features[7] = math.log(1 + np.sum(Co_occur_fre_all.sum(axis=0) * min_dist_all) / (Co_occur_fre_all.sum() + 0.001), 10)

    return features


def mAP_differ(term, query_id, docIds, idf, mAP):
    """
    Args:
        term: str
        query_id: str
        docIds: str list
        idf: float
    Returns:
        differ: float
    """
    global query_dict, doc_dict, rele_dict

    # calculate bm25 scores
    docScore = dict()
    for docId in docIds:
        query = example_list2dict(query_dict[query_id] + ' ' + term)
        doc = example_list2dict(doc_dict[docId])
        docScore[docId] = bm25(query, doc, idf)
      
    # sort
    labels = []
    sort = sorted(docScore.items(), key=lambda item: item[1], reverse=True)
    for i in range(0, len(docIds)):
        labels.append(int(rele_dict[query_id][sort[i][0]]))
    labels = np.array(labels)
    # calc recall and precision
    recall = []
    precis = []
    Tp_Fn = np.sum(labels)
    for N in range(0, labels.shape[0]):
        Tp_Fp = N + 1
        Tp = np.sum(labels[0:N + 1])
        recall.append(Tp / Tp_Fn)
        precis.append(Tp / Tp_Fp)
    recall = np.array(recall)
    precis = np.array(precis)

    mAP_expan = ave_prec(recall, precis, True)
    differ = mAP_expan - mAP

    return differ

In [7]:
train_x = np.empty(shape=[0, 8])
train_y = np.empty(shape=[0, 1])

idf = cal_idf(doc_dict=doc_dict)

for query_id in range(50, 249):
    print('-----Current query ID: %d' % query_id)
    # docs'ID corresponding to query_id
    doc_ids = []
    docs = []
    for Id in range(0, 300):
        st = '%s_%d' % (query_id, Id)
        if doc_dict.get(st) is None:
            break
        else:
            doc_ids.append(st)
            docs.append(doc_dict[st].split())
    
    # calculate bm25 scores of docs
    doc_score = dict()
    query = example_list2dict(query_dict[str(query_id)])
    for doc_id in doc_ids:
        doc = example_list2dict(doc_dict[doc_id])
        doc_score[doc_id] = bm25(query, doc, idf)

    # sort and choose top N docs
    # get the sorted label
    N = 20
    top_N = []
    labels = []
    sort = sorted(doc_score.items(), key=lambda item: item[1], reverse=True)
    for i in range(0, len(doc_ids)):
        if i < N:
            top_N.append(sort[i][0])
        labels.append(int(rele_dict[str(query_id)][sort[i][0]]))
    labels = np.array(labels)

    # calc recall and precision
    recall = []
    precis = []
    Tp_Fn = np.sum(labels == 1)
    for N in range(0, labels.shape[0]):
        Tp_Fp = N + 1
        Tp = np.sum(labels[0:N + 1] == 1)
        recall.append(Tp / Tp_Fn)
        precis.append(Tp / Tp_Fp)
    recall = np.array(recall)
    precis = np.array(precis)
    # calc mean average precision
    mAP = ave_prec(recall, precis)
    # candidate expansion terms
    candidate = []
    for doc_id in top_N:
        candidate.extend(doc_dict[doc_id].split())
    candidate = list(set(candidate) - set(query_dict[str(query_id)].split()))

    # features vector and label for each expansion term
    X = []
    for term in candidate:
        mAP_d = mAP_differ(term, str(query_id), doc_ids, idf, mAP)
        if mAP_d > 0.1:
            X.append(get_feature(term, str(query_id), doc_ids, top_N))
            train_y = np.append(train_y, [[1]], axis=0)
        elif mAP_d < -0.1:
            X.append(get_feature(term, str(query_id), doc_ids, top_N))
            train_y = np.append(train_y, [[0]], axis=0)
    X = np.array(X)
    if X.shape[0] > 0:
        X = normalize(X, axis=0, norm='max')
        train_x = np.append(train_x, X, axis=0)
# ------------------------------------------
# save train data
np.savetxt('E:/PycharmProjects/EE448/ee448-project2-query-expansion/train_x.txt', train_x)
np.savetxt('E:/PycharmProjects/EE448/ee448-project2-query-expansion/train_y.txt', train_y)
# ------------------------------------------

-----Current query ID: 50
37


KeyboardInterrupt: 

In [ ]:
train_x = np.loadtxt('E:/PycharmProjects/EE448/ee448-project2-query-expansion/train_x.txt')
train_y = np.loadtxt('E:/PycharmProjects/EE448/ee448-project2-query-expansion/train_y.txt')

train_y = train_y.astype(int)
train_y.reshape(train_y.shape[0], 1)

x_train, x_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.3)

classifier = svm.SVC(C=10, kernel='rbf', gamma='auto', class_weight='balanced', probability=True)
classifier.fit(train_x, train_y)

print("train accuracy:", classifier.score(x_train, y_train) )
print("test  accuracy:", classifier.score(x_test, y_test) )

query_expan = dict()
#idf = cal_idf(doc_dict=doc_dict)

for query_id in range(0, 50):
    print('-----Current query ID: %d' % query_id)

    # docs'ID corresponding to query_id
    doc_ids = []
    for Id in range(0, 300):
        st = '%s_%d' % (query_id, Id)
        if doc_dict.get(st) is None:
            break
        else:
            doc_ids.append(st)
    
    # calculate bm25 scores of docs
    doc_score = dict()
    query = example_list2dict(query_dict[str(query_id)])
    for doc_id in doc_ids:
        doc = example_list2dict(doc_dict[doc_id])
        doc_score[doc_id] = bm25(query, doc, idf)
        
    # sort and choose top N docs
    N = 20
    top_N = []
    sort = sorted(doc_score.items(), key=lambda item: item[1], reverse=True)
    for i in range(0, N):
        top_N.append(sort[i][0])

    # candidate expansion terms
    candidate = []
    for doc_id in top_N:
        candidate.extend(top_tfidf(doc_dict[doc_id].split(), 20))
    candidate = list(set(candidate) - set(query_dict[str(query_id)].split()))
    candidate.sort()
    
    features = []
    for term in candidate:
        feature = get_feature(term, str(query_id), doc_ids, top_N)
        features.append(feature)
        
    features = np.array(features)
    features = normalize(features, axis=0, norm='max')
    pred = classifier.predict_proba(features)
    
    labels = dict()
    n = 0
    for label in pred:
        if label[1] > 0.6:
            labels[candidate[n]] = label[1]
        n += 1
           
    label_sort = sorted(labels.items(), key=lambda item: item[1], reverse=True)

    tmp = ''
    count = 20
    for item in label_sort:
        if count == 0:
            break
        if tmp == '':
            tmp += item[0]
        else:
            tmp = tmp + ' ' + item[0]
        count -= 1
        
    query_expan[str(query_id)] = tmp

# ------------------------------------------
# save expanded query
file_handle = open( 'E:/PycharmProjects/EE448/ee448-project2-query-expansion/query_e.txt', mode='w')
for query_id in range(0, 50):
    file_handle.write(str(query_id)+'\t'+query_dict[str(query_id)]+' '+query_expan[str(query_id)]+'\n')
file_handle.close()
# ------------------------------------------